##### Imports

In [0]:
from pyspark.sql import functions as F, types as T
from pyspark.sql.avro.functions import from_avro
from cryptography.fernet import Fernet
import base64

##### Variables

In [0]:

eh_namespace = 'evhnscjprd001'
eventhub_name = "evh_user_random_schema"
connection_string = dbutils.secrets.get(scope="data-master-akv", key="EventhubConnectionString")
storage_account_name  = "stacjprd001"
database_name = "bronze_users"
table_name = "users" 
container_name = "cont-dt-mst"
container_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/bronze/{database_name}"
delta_table_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/bronze/{database_name}/{table_name}"

with open("/Workspace/Users/cristianosjunior2012@hotmail.com/user_schema.avsc", "r") as file:
    schema_json = file.read()

eh_conf = {
   'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(f"{connection_string};EntityPath={eventhub_name}"),
    'eventhubs.startingPosition': '{"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}'
}


##### Function Encrypt

In [0]:
key = Fernet.generate_key()
fernet = Fernet(key)

def encrypt_data(data):
    return fernet.encrypt(data.encode()).decode() if data else None

spark.udf.register("encrypt_data", lambda z: encrypt_data(z))

<function __main__.<lambda>(z)>

##### Read Stream

In [0]:
df = (spark.readStream
    .format("eventhubs")
    .options(**eh_conf)
    .load())
df.printSchema()

root
 |-- body: binary (nullable = true)
 |-- partition: string (nullable = true)
 |-- offset: string (nullable = true)
 |-- sequenceNumber: long (nullable = true)
 |-- enqueuedTime: timestamp (nullable = true)
 |-- publisher: string (nullable = true)
 |-- partitionKey: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- systemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



##### Edit columns

In [0]:
df = df.select(from_avro(F.col("body"), schema_json).alias("data")) \
    .select('data.*') \
    .withColumn('ingest_time', F.current_timestamp())\
    .withColumn('dat_ref_carga', F.to_date('ingest_time')) 

for col in df.columns:
    df = df.withColumn(col, F.col(col).cast('string'))

df = df.withColumn("password", F.expr("encrypt_data(password)")) \
    .withColumn("email", F.expr("encrypt_data(email)")) \
    .withColumn("phone_number", F.expr("encrypt_data(phone_number)")) \
    .withColumn("credit_card", F.expr("encrypt_data(credit_card)"))

schema_df = ', '.join(f"{col} string" for col in df.columns)

##### Create Database

In [0]:
spark.sql(f"""CREATE DATABASE IF NOT EXISTS {database_name} LOCATION '{container_path}'""")

spark.sql(f"""CREATE TABLE IF NOT EXISTS {database_name}.{table_name} (
            {schema_df}
        )
    USING DELTA
    PARTITIONED BY (dat_ref_carga)
    LOCATION '{delta_table_path}'
    """)

DataFrame[]

##### Write table

In [0]:
query = (df.writeStream
         .format("delta")
         .outputMode("append")
         .trigger(once=True)
         .option("checkpointLocation", f"{delta_table_path}/_checkpoints/")
         .start(delta_table_path))

try:
    query.awaitTermination()
except Exception as e:
    print(f"Consulta falhou: {e}")

##### Output

In [0]:
%sql
select * from bronze_users.users

id,uid,password,first_name,last_name,username,email,avatar,gender,phone_number,social_insurance_number,date_of_birth,employment,address,credit_card,subscription,ingest_time,dat_ref_carga
9479,bb0eb365-c212-4ff9-bc60-6ee3693cbb20,gAAAAABnreZzOtr8dWRhzMuGfSf7XJhIi2L5AVQi1P5gpJSlj5SvlyNPP0CQWihBd0s9D4lp9tWet5bM9ndhmROf6CObE2xBkA==,Keeley,Schmeler,keeley.schmeler,gAAAAABnreZzBTBx6CQOWax5DezA7mYbdhrAWgMtvCrOm9C2jXrPZB6yphbmwVUkZJOP32rVUbLj61_Cc7mqS4vLq9WC2KWn8Nzp1POHwc62rhcXiFmhY8E=,https://robohash.org/voluptascumanimi.png?size=300x300&set=set1,Agender,gAAAAABnreZzpgWuhSxKnt4a2JXQ8gY469Qu-erGit2CdgilkweTkVg7xMGdgSCPx1HBSQSwgd4iYeVY2JsTDEsdOw3pZrgS7fLDGY5FEqzGBzQa6K7JlW8=,541144085,1966-10-22,"{Consulting Manager, Teamwork}","{Hildredburgh, Shanahan Wall, 8011 Britt Track, 94861-6824, Indiana, United States, {22.06699, 125.46814}}",gAAAAABnreZzSDZPWdYwqsLVF1ENKLoYhGoTTeNv0tlHQ4qW22TRxIUJUfJPVZnSOxzZ6lXM-OmclWGLfgcKaBva-hsjq6qTxZmX5fUJielvXKtbhhGjLvU=,"{Bronze, Active, Bitcoins, Full subscription}",2025-02-13 12:32:47.822,2025-02-13
9846,53079e09-422e-4dbb-99b6-c3c7c4342b21,gAAAAABnreZzjx8Zi0ter9SoUxyMPcJXr_roCPIE5wutbITsEfcmo3104ocSmNnNo968voJZ-GiNAPrdZyDN5wgedzDtzi0s-g==,Kenton,Bayer,kenton.bayer,gAAAAABnreZzMmhmrGeEvdxnIUd6fOlmIsq4Qx2B6GdlTrAwcdFIXL0JY5ccW1ZwEZxVNfwTbf0bcy8tHZdYPz-VjZ0Jc75aNrxPzbS1R6AFPmjcP8_zhNQ=,https://robohash.org/eumnihilvoluptatem.png?size=300x300&set=set1,Agender,gAAAAABnreZzQNJMjG8f4Y9s_JsCRfYtvrPN_JnTKoydGcZKuqCYJsgRxj2OJDA00_yxmTbXgfpMs0r9mYnNeviRg2BKilhMYboMmzmS90lofb7vK8ieG60=,418950937,2000-07-19,"{Senior Education Representative, Problem solving}","{Port Josh, George Parkways, 16585 Kent Forges, 20634-8646, Nebraska, United States, {-72.40323, -138.2184}}",gAAAAABnreZzw1mgYfJYUyhKQx-rznXnH_yBYCx6qyFwxeeMxfBriXPquBCYrtsNvJqbFLs01JYofwhe2Axz1LtDHIRD1zJCw-CvFLXm6eFZ5dyM7h0ZBQg=,"{Essential, Idle, Cash, Full subscription}",2025-02-13 12:32:47.822,2025-02-13
4745,ba4af051-26f8-4c5e-bb91-743bb50aa20e,gAAAAABnreZzoQNuaoSdvzBFE5DCn6YomHzsNl300EkDaji7C7GxkrgBj5HUAHsKrDBRLBiC3bJ4VAbCW9blJCCKF8LfZ7MA0g==,Maryanne,Considine,maryanne.considine,gAAAAABnreZzgmLtOI2-Jmcu0uG4KlG37_0k5-Z-JYCl0rDZldWZKw1SWUWCqhQOkw9tqIG9IybaGTG6KjJrbzIOumHk0qwo6rORTW-grJ8reTRf2fodRQc=,https://robohash.org/hicnobisut.png?size=300x300&set=set1,Polygender,gAAAAABnreZzdsNCgdGTzLoQ3aKjrHIQhjNBEFYc-Q2PxQyfSqTNZ7trm0Vuu_jxaH4sIa_Te5urzgwSqCTm2sdKGxi7pvX8Ej34Ad_ftzNjbsJ8TVffHiA=,681502621,1968-09-24,"{Retail Liaison, Networking skills}","{Jeffreyport, Janell Ways, 1968 Cartwright Coves, 83746, Iowa, United States, {56.485775, -126.37058}}",gAAAAABnreZzvf0M3SNJx5MVTNACfeu9Gkctk_9u12O2xATd6LvLaV_y6TYFAdQi8b8DJhFK8yv3hS_3kaf86T4yvZSwJ_IhjjL9tTCXwibAvZdceAAGqOg=,"{Business, Blocked, Apple Pay, Monthly}",2025-02-13 12:32:47.822,2025-02-13
8164,7f2c97aa-492c-4039-a1cc-7d05cfde6f6f,gAAAAABnreZz9I8PtQmBv7VBK90UYzoiPs8BDbKHWzk_hAgL5ovaJSCVuDcFJu15EKsPO5Jpmvr1l-qij3_6h2ogScl0QhCa9Q==,Colin,Hansen,colin.hansen,gAAAAABnreZzlBiA4RKfTvosDgcp1N_cLVhJdQCb2lk0n9oxmp_XP8yYDHL8bKJNtXYsq_5J0yqv_ugYPsksZyCw2fDCIbCZ_8N2YZN5XkatdKfpgQwaBi0=,https://robohash.org/porroinaut.png?size=300x300&set=set1,Male,gAAAAABnreZzSi4DzCr-zzrXED4BMg0aKs1XszEwXDbIxj8jBg25q9STPW3NUtJHiNC_sn4GvNW1CrVEVkybRIc5wFisE3eZgPXs_57A81qcrJknXgcMH7E=,178662409,1964-06-27,"{Regional Government Supervisor, Self-motivated}","{Harlanhaven, Becker Harbors, 81791 Roberts Lock, 93725, Pennsylvania, United States, {-52.499435, 167.17781}}",gAAAAABnreZzBLNfBZathM-DPs1oPotJwfujrB4Y8ZXumJYXw_fmqixcOI00D6V4xcofhK0NsHLhSpnP_1xc_eOxCIGcrmy8bnmu7-tOPjTlbs-v-JiwmX0=,"{Essential, Idle, Google Pay, Payment in advance}",2025-02-13 12:32:47.822,2025-02-13
8001,d9e03bb1-9ca0-419e-ac78-d13642b8f90a,gAAAAABnreZz1BkuCAp48cGqDkrypbqJvlQopKBk2bdhqYWRM6_syQ46NjylHL3wUaZe8mP0qw9HyMqbW3SXVblaxDv0xTxBLA==,Guy,Hayes,guy.hayes,gAAAAABnreZzra93F6Nl--rwOWUDhuROhD7Vb2EMs_uzpDuwz_Kax_qS4J4lXgN3wpJArSR1Ol4WVeeSBV01GLRM3L0vHyeT5PeSXDILlDzTXeEh42sPMu4=,https://robohash.org/molestiaesintreprehenderit.png?size=300x300&set=set1,Polygen